In [32]:
import pandas as pd
import networkx as nx

In [34]:
data = pd.read_csv('data/Denmark/Merged.csv')

,Unnamed: 0,typeid_x,afstemningid,aktørid,opdateringsdato_x,id_y,typeid_y,navn,opdateringsdato_y,startdato,slutdato,party
0,0,1,6922,221,2019-01-15T16:31:11.32,221,5,Pia Adelsteen,2024-12-11T12:49:52.567,2010-10-06T00:00:00,NaN,Dansk Folkeparti
1,1,1,6921,221,2019-01-16T10:45:19.98,221,5,Pia Adelsteen,2024-12-11T12:49:52.567,2010-10-06T00:00:00,NaN,Dansk Folkeparti
2,2,2,6923,221,2019-01-22T13:02:09.41,221,5,Pia Adelsteen,2024-12-11T12:49:52.567,2010-10-06T00:00:00,NaN,Dansk Folkeparti
3,3,1,6924,221,2019-01-24T10:02:15.453,221,5,Pia Adelsteen,2024-12-11T12:49:52.567,2010-10-06T00:00:00,NaN,Dansk Folkeparti
4,4,1,6925,221,2019-01-24T10:03:04.13,221,5,Pia Adelsteen,2024-12-11T12:49:52.567,2010-10-06T00:00:00,NaN,Dansk Folkeparti
...,...,...,...,...,...,...,...,...,...,...,...,...
302093,302093,1,10110,21044,2025-01-16T17:00:16.147,21044,5,Lasse Haugaard Pedersen,2025-02-03T09:39:35.91,2024-12-01T00:00:00,NaN,Socialdemokratiet
302094,302094,1,10111,21044,2025-01-23T10:01:05.52,21044,5,Lasse Haugaard Pedersen,2025-02-03T09:39:35.91,2024-12-01T00:00:00,NaN,Socialdemokratiet
302095,302095,1,10112,21044,2025-01-23T10:01:33.953,21044,5,Lasse Haugaard Pedersen,2025-02-03T09:39:35.91,2024-12-01T00:00:00,NaN,Socialdemokratiet
302096,302096,1,10113,21044,2025-01-23T10:02:11.817,21044,5,Lasse Haugaard Pedersen,2025-02-03T09:39:35.91,2024-12-01T00:00:00,NaN,Socialdemokratiet


In [173]:
#dict, key = id, values = (afstemningid, typeid)
# dict of all people with all their votes (voting id and what they voted)

PA = {}          #<<-------------- Person, Afstemning
for idx, row in data.iterrows():
    if row['aktørid'] not in PA:
        PA[row['aktørid']] = [(row['afstemningid'])]
    else:
        PA[row['aktørid']].append((row['afstemningid']))

PAV = {}          #<<-------------- Person, Afstemning, Vote
for idx, row in data.iterrows():
    if row['aktørid'] not in PAV:
        PAV[row['aktørid']] = [(row['afstemningid'],row['typeid_x'])]
    else:
        PAV[row['aktørid']].append((row['afstemningid'],row['typeid_x']))


PAVP = {}          #<<-------------- Person, Afstemning, Vote, Party
for idx, row in data.iterrows():
    if row['aktørid'] not in PAVP:
        PAVP[row['aktørid']] = [(row['afstemningid'],row['typeid_x'],row['party'],row['navn'])]
    else:
        PAVP[row['aktørid']].append((row['afstemningid'],row['typeid_x'],row['party'],row['navn']))

#dict of parties with memberid as values

PP = {}          #<<-------------- Party, Person
for idx, row in data.iterrows():
    if row['party'] not in PP:
        PP[row['party']] = [(row['aktørid'])]
    else:
        PP[row['party']].append((row['aktørid']))


In [ ]:
#we check both ways, we calc both i -> j and j->i.
# person 221 as i and 1619 as j, and the other way. 1619 as i and 221 as j. is that a problem? 
#there are prob some variables that needs renaming

def calc_thres(PP, PA, PAV):
    party_agree = []
    for party in PP:
        print(party)
        memb_agree = []
        for member1 in set(PP.get(party)):
            for member2 in set(PP.get(party)):
                if member1 != member2:
                    common = set(PAV.get(member1)) & set(PAV.get(member2)) # how many votes do they agree on 
                
                    common2 = set(PA.get(member1)) & set(PA.get(member2)) # how many same bills have they voted on

                    if len(common) != 0: # if they have at least one vote thats the same, (too avoid divide by 0)
                    
                        percentage = len(common) / len(common2)
                        memb_agree.append(percentage)
                    else:
                        memb_agree.append(0)
        if len(memb_agree) != 0:
            avgMemb = sum(memb_agree)/len(memb_agree)
            party_agree.append(avgMemb)
            print(avgMemb)
        else:
            print(party) # party where there is only one member
            print('1')
            #party_agree.append(1)
    avgParty = sum(party_agree)/len(party_agree)
    print(max(party_agree))
    return avgParty

res = calc_thres(PP,PA,PAV)
res

In [194]:
def edgelist_calc(PA, PAV, threshold):
    edgelist=[]
    for i in PAV: # person 1

        for j in PAV: # for loop for person 2

            if i != j: # checks they are not the same

                common = set(PAV.get(i)) & set(PAV.get(j)) # how many votes do they agree on 
                
                common2 = set(PA.get(i)) & set(PA.get(j)) # how many same bills have they voted on

                if len(common) != 0: # if they have at least one vote thats the same, (too avoid divide by 0)
                    
                    percentage = len(common) / len(common2)
                    if percentage > threshold:
                        edgelist.append((i,j))
                

    return list(set(edgelist))
    

In [196]:
edgelist = edgelist_calc(PA,PAV, res)
G_denmark = nx.from_edgelist(edgelist)

In [183]:
len(edgelist)

76572

In [185]:
list_of_dangle = [node for node in G_denmark.nodes if G_denmark.degree(node) == 0]
list_of_dangle

[]